In [1]:
import vectorbtpro as vbt
import numpy as np
import pandas as pd

vbt.settings.set_theme('dark')
vbt.settings['plotting']['layout']['width'] = 800
vbt.settings['plotting']['layout']['height'] = 400

In [2]:
df = pd.read_csv('2ySOLdata1h.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
df.set_index('timestamp', inplace=True)

In [3]:
data = vbt.Data.from_data(df)
data

In [157]:
# data.plot().show()

In [4]:
open_price = data.get('Open')
high_price = data.get('High')
low_price = data.get('Low')
close_price = data.get('Close')

In [ ]:
# vbt.phelp(vbt.BBANDS.run)

In [ ]:
adx = vbt.ADX.run(high_price, low_price, close_price, window=14)
atr = vbt.ATR.run(high_price, low_price, close_price, window=14)
bbands = vbt.BBANDS.run(close_price, window=14)
rsi = vbt.RSI.run(close_price)
sma = vbt.MA.run(close_price, window=20)
strend = vbt.SUPERTREND.run(high_price, low_price, close_price, period=7, multiplier=3)

data.data['symbol']['ADX'] = adx.adx
data.data['symbol']['ATR'] = atr.atr
data.data['symbol']['BBAND'] = bbands.bandwidth
data.data['symbol']['RSI'] = rsi.rsi
data.data['symbol']['SMA'] = sma.ma
data.data['symbol']['STREND'] = strend.trend
data.data['symbol'] = data.data['symbol'].dropna()
predictor_list = ['Close', 'ADX', 'ATR', 'BBAND', 'RSI', 'SMA', 'STREND']

X = data.data['symbol'][predictor_list]

y = data.data['symbol']['signal']

In [55]:
from sklearn.model_selection import train_test_split
split_percentage = 0.7
split = int(split_percentage*len(X))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1-split_percentage)


In [135]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

clf = BaggingClassifier(
        estimator = DecisionTreeClassifier(criterion='gini', max_depth=10, min_samples_leaf=5),
        n_estimators = 10,
        random_state = 42
    ).fit(X_train, y_train)

y_pred = clf.predict(X_test)

In [136]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
report = classification_report(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

In [137]:
df_split = data.data['symbol'][split:].copy()
df_split.loc[:, "signal"] = y_pred
signal = df_split['signal']
entries = signal == 'SignalLong'
exits = signal == 'SignalShort'

In [138]:
pf = vbt.Portfolio.from_signals(
    close=df_split.Close, 
    long_entries=entries, 
    long_exits=exits,
    size=100,
    size_type='value',
    init_cash='auto'
)

In [139]:
pf.plot(settings=dict(bm_returns=False)).show()

In [140]:
pf.stats()

Start                         2023-05-26 21:00:00+00:00
End                           2023-12-31 06:00:00+00:00
Period                                218 days 10:00:00
Start Value                                       100.0
Min Value                                     98.836553
Max Value                                    342.182467
End Value                                    305.947321
Total Return [%]                             205.947321
Benchmark Return [%]                         421.028013
Total Time Exposure [%]                       46.127432
Max Gross Exposure [%]                            100.0
Max Drawdown [%]                              13.497222
Max Drawdown Duration                  36 days 04:00:00
Total Orders                                          4
Total Fees Paid                                     0.0
Total Trades                                          2
Win Rate [%]                                      100.0
Best Trade [%]                               137

In [141]:
# import joblib
# def save_model(model, model_filename='trained_model.joblib'):
#     joblib.dump(model, model_filename)
#     print(f'Model saved as {model_filename}')
# save_model(clf)